In [1]:
import numpy as np
import sys
import re
import os
import cygnus_lib as cy
import toolslib as tl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
#import mylib as my
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line, resize, rescale)
import scipy.ndimage
from skimage.transform import hough_circle, hough_circle_peaks
from scipy.stats import norm
import scipy.stats as stats
from iDBSCAN import iDBSCAN


import pandas as pd
import pickle
import time
from IPython import display

%matplotlib inline
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
plt.rcParams['font.serif'] = ['Bitstream Vera Sans']
x_resolution = y_resolution = 2048
from itertools import combinations

In [ ]:
t0 = time.time()

cy.set_atlas_style('large')
############# Data in imput #######################################
dataSelection = 'FNG'
#dataSelection = 'FNG'

#runI = [494, 492, 493]
runI = [815]
#runI = [813,814]

debug  = False
files = ""
################ analysis cards ################################
nsigma       = 1         # numero di sigma sopra il piedistallo
cimax        = 200       # valori del cut sull'imagine
iTr          = 0        # traccia di partenza
rescale      = 512       # binnagio finale immagine (deve essre un sottomultipli della 2**2 risluzione di partenza)
minClose     = 2         # minimum cluser size (rebinne image)
eps          = 5         # maximum distance for the cluster point
maxClose     = 30000     # massima dimesione del clustr evita le scriche      
Cmethod      = 'idbsc'    #'hdbs' # 'nccs' # 'dbsc' # 'idbsc'
max_image_to_read = 60  # 0 all
############### Inzializzazione varibili e costanti #################
scale        = int(x_resolution/rescale)



iterative    = 4         # number of iterations for the IDBSC
vector_eps = [2, 2.9, 3.6, 5]#[2.26, 3.5, 2.8, 6]
vector_min_samples = [2, 15, 10, 2]#[2, 30, 6, 2]
cuts = [0, 0]#[900, 150]


# MAIN LOOP ON 

data    = [] # output data
 
for nRi in range(0,len(runI)):
    
    max_image=np.size(cy.swift_listdir(cy.file2FullPathCygnus(dataSelection, runI[nRi], 'H5')))
    print ("# of Image Files: %d" % (max_image))
    if max_image_to_read != 0:
        max_image = max_image_to_read   
    print ("WARNING: data will be anlyzed from %d to %d" % (iTr, max_image))

    if max_image == 0:
        print ("No file or image for file %s" % runI[nRi])
        print ("STOP")
        break

    #
    # load pedestal value generated by runs-pedestals.ipynb script
    # 
    try:
        fileoutm = ("./data/run817_mean.h5")
        m_image = cy.read_image_h5(fileoutm)
        PedOverMax = m_image[m_image > cimax].size
        print ("Pedestal mean: %.2f, sigma: %.2f, over th. (%d) %d" % 
           (m_image[m_image<cimax].mean(), 
            np.sqrt(m_image[m_image<cimax].var()), cimax,
            (m_image>cimax).sum()))
    except:
        print ("No Pedestal file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
        print ("STOP")
        break
    try: 
        fileouts = ("./data/run817_sigma.h5")
        s_image = cy.read_image_h5(fileouts)
        print ("Sigma mean: %.2f, sigma: %.2f, over th. (50) %d" % 
       (s_image[s_image<50].mean(), 
        np.sqrt(s_image[s_image<50].var()), 
        (s_image>50).sum()))
    except:
        print ("No Sigma file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
        print ("STOP")
        break
                
    #
    # Run by run init 
    #
    th_image   = np.round(m_image + nsigma*s_image) # verficare con il np.round.... np.ceil
    # th_image[:,:]=101 # per imostare tutto a 101
    
    TrOk   = 0
    dCloseT= 0

    #################################
    # MAIN LOOP ON RUN
    ##################################
    
    for iTr in range(0, max_image):
        # load image
        image = cy.swift_read_image_h5(cy.imageFile2FullPathCygnus(dataSelection, runI[nRi], iTr)) 
        
        if (np.sum(image)/np.sum(m_image)) < 1.09: # Avoid wrong images 1.02 with we want to skip sparks too
            if image[image> cimax].size<1000000: # limit on the number og pixel over high cutof
                TrOk += 1
                print ("Analyzing Run %d Image %d \n" % (runI[nRi],iTr))
                print ("SumLight/SumPedestal = %.4f" % (np.sum(image)/np.sum(m_image)))
            
                edges_image     = (image > th_image) & (image < cimax)
                rebin_image     = cy.rebin(image, (rescale, rescale))
                rebin_th_image  = cy.rebin(th_image, (rescale, rescale))
                edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
                points          = np.array(np.nonzero(edges)).T.astype(float)
                
                clusters = iDBSCAN(iterative = iterative, vector_eps = vector_eps, vector_min_samples = vector_min_samples, cuts = cuts).fit(points)
                
                n_clusters_ = len(set(clusters.labels_)) - (1 if -1 in clusters.labels_ else 0)
                
                for Ci in range (0, n_clusters_): # number of clusters found in the image
                    cluvar   = [] # cluster info
                    #print ("Salving cluster %d from %d" % (Ci,n_clusters_))
                    Xi, Yi, Is, Ib, tag = tl.cluInfo(clusters,points,Ci,image,th_image,scale) # extract cluster information
                    # cluvar = [Run, Imag, Tag, X position, Y position, Light signal, Light pedestal]
                    cluvar.append(runI[nRi])
                    cluvar.append(iTr)
                    cluvar.append(tag)
                    cluvar.append(Xi)
                    cluvar.append(Yi)
                    cluvar.append(Is)
                    cluvar.append(Ib)
                    data.append(cluvar) # save the cluster info in a List
        print ("...Next image... \n")
t1 = time.time()
print ("Elapsed time of the algorithm: %.2f seconds / %.2f minutes" % ((t1-t0),(t1-t0)/60))

# of Image Files: 300
Pedestal mean: 99.19, sigma: 0.77, over th. (200) 10
Sigma mean: 2.48, sigma: 2.12, over th. (50) 11
Analyzing Run 815 Image 0 

SumLight/SumPedestal = 1.0007
...Next image... 

Analyzing Run 815 Image 1 

SumLight/SumPedestal = 1.0002
...Next image... 

Analyzing Run 815 Image 2 

SumLight/SumPedestal = 1.0007
...Next image... 

Analyzing Run 815 Image 3 

SumLight/SumPedestal = 1.0001
...Next image... 

Analyzing Run 815 Image 4 

SumLight/SumPedestal = 1.0006
...Next image... 

Analyzing Run 815 Image 5 

SumLight/SumPedestal = 1.0003
...Next image... 

Analyzing Run 815 Image 6 

SumLight/SumPedestal = 1.0006
...Next image... 

Analyzing Run 815 Image 7 

SumLight/SumPedestal = 1.0005
...Next image... 

Analyzing Run 815 Image 8 

SumLight/SumPedestal = 1.0005
...Next image... 

Analyzing Run 815 Image 9 

SumLight/SumPedestal = 1.0004
...Next image... 

Analyzing Run 815 Image 10 

SumLight/SumPedestal = 1.0005
...Next image... 

Analyzing Run 815 Image 11 



## Salving table

In [ ]:
t2 = time.time()
#------------------- Saving File ------------------------------#
directory = ("./data/")          # Directory of the output file
filename  = ("ClustersTable_%d" % (runI[0]))    # Name of the output file
extension = (".csv")             # Extension of the output file
dataout = directory + filename + extension # Full path of the output file

import pandas as pd

columns = ["Run","Image","Tag","X","Y","Light","Pedestal"]
df = pd.DataFrame(data, columns = columns)

#Now you have a csv with columns and index:
df.to_csv(dataout,index=False)

t3 = time.time()
print ("Table saving time: %.2f seconds / %.2f minutes" % ((t3-t2),(t3-t2)/60))